# TODO:
- [x] UPDATEN VAN DE bag_remove KOLOMMEN
- [x] BASE NOTEBOOK UPDATEN & MASTER NOTEBOOK HIERIN AANROEPEN (PAPERMILL)
- [x] MACHINE LEARNING WEER LATEN WERKEN
- [x] DOCUMENTEREN NOTEBOOKS
- [ ] DOCUMENTEREN README
- [ ] RandomForestRegressor gebruiken ipv RandomForestClassifier (heeft andere evaluation metric nodig, of ergens instellen dat 50+% confidence == True, en 50-% confidence == False.
- [ ] Checken of droppen van text kolommen uit zaken df, net voorafgaand aan de train/test set creation, okay is. 
- [ ] PARAMETER OPTIMALISATIE SEARCH RUNNEN

In [25]:
# Load public modules.
import os, sys
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from pathlib import Path
import time

# Get the home dir and username.
HOME = Path.home()
USERNAME = os.path.basename(HOME)

# Set codebase path for old VAO.
CODEBASE_PATH_OLD = os.path.join(HOME, 'Documents/woonfraude/codebase/')
sys.path.insert(1, CODEBASE_PATH_OLD)
                
# Set codebase path for new VAO.
CODEBASE_PATH_NEW = os.path.join('/data', USERNAME, 'Documents/woonfraude/codebase/')
sys.path.insert(1, CODEBASE_PATH_NEW)

# Import own modules.
from datasets_oo import *
from clean_oo import *
from build_model import *
from extract_features_oo import *

In [2]:
# Set global variables.
FORCE_DOWNLOAD = False
FORCE_DATASET_SPECIFIC_PREPROCESSING = False

# Load all datasets in memory

In [3]:
##############################
# Initialize dataset objects #
##############################

adresDataset = AdresDataset()
zakenDataset = ZakenDataset()
stadiaDataset = StadiaDataset()
personenDataset = PersonenDataset()
bagDataset = BagDataset()
hotlineDataset = HotlineDataset()

In [4]:
#######################################################################################
# Download data and perform dataset-specific pre-processing steps for dataset objects #
#######################################################################################

# Forces the downloading of new data.
if FORCE_DOWNLOAD:
    adresDataset.download(force=True)
    zakenDataset.download(force=True)
    stadiaDataset.download(force=True)
    personenDataset.download(force=True)
    bagDataset.download(force=True)
    

 # Forces the dataset specific pre-processing of the downloaded data.
if FORCE_DATASET_SPECIFIC_PREPROCESSING:
    
    # Adres dataset.
    adresDataset.load('download')
    adresDataset.extract_leegstand()
    adresDataset.enrich_with_woning_id()

    # Zaken Dataset.
    zakenDataset.load('download')
    zakenDataset.add_categories()
    zakenDataset.filter_categories()  # Verwijder meldingen met categorieeen "woningkwaliteit" en "afdeling vergunningen en beheer".

    # Stadia dataset.
    stadiaDataset.load('download')
    stadiaDataset.add_zaak_stadium_ids()
    stadiaDataset.add_labels()

    # Bag dataset.
    bagDataset.download(force=True)
    bagDataset.load('download')
    bagDataset.bag_fix()

    # Hotline dataset.
    hotlineDataset.download(force=True)

In [5]:
###################
## Load datasets ##
###################
# Load datasets from cache (when download and pre-processing steps in previous block have been done).

adresDataset.load('download_leegstand_woningId')
zakenDataset.load('download_categories_filterCategories')
stadiaDataset.load('download_ids_labels')
personenDataset.load('download')
bagDataset.load('download_columnFix')
hotlineDataset.load('download')

Version 'download_leegstand_woningId' of dataset 'adres' loaded!
Version 'download_categories_filterCategories' of dataset 'zaken' loaded!
Version 'download_ids_labels' of dataset 'stadia' loaded!
Version 'download' of dataset 'personen' loaded!
Version 'download_columnFix' of dataset 'bag' loaded!
Version 'download' of dataset 'hotline' loaded!


# Clean and extract features from all datasets

In [6]:
######################################################
## Clean zaken dataset & perform feature extraction ##
######################################################

zakenPipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=zakenDataset.id_column,
        drop_duplicates=True,
        fix_date_columns=['begindatum','einddatum', 'wzs_update_datumtijd'],
        clean_dates=True,
        lower_string_columns=True,
        impute_missing_values=True)
    ),
#     ('extract', FeatureExtractionTransformer(
#         categorical_cols_hot=['afg_code_beh', 'beh_code', 'eigenaar', 'categorie'])
#     )
    ])

zaken = zakenPipeline.fit_transform(zakenDataset.data)

Dataframe "zaken": Dropped 0 duplicates!
Dataframe "zaken": Fixed dates!
Dataframe "zaken": Cleaned out 3 dates!
Lowered strings of cols ['beh_code', 'beh_oms', 'afg_code_beh', 'afs_code', 'afs_oms', 'afg_code_afs', 'eigenaar', 'zaak_id', 'mededelingen', 'categorie'] in df zaken!
Missing values in df zaken have been imputed!


In [7]:
##########################
## Clean stadia dataset ##
##########################

stadiaPipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=stadiaDataset.id_column,
        drop_duplicates=True,
        fix_date_columns=['begindatum', 'peildatum', 'einddatum', 'date_created',
                          'date_modified', 'wzs_update_datumtijd'],
        clean_dates=True,
        lower_string_columns=True,
        impute_missing_values=True)
    )])

stadia = stadiaPipeline.fit_transform(stadiaDataset.data)

Dataframe "stadia": Dropped 0 duplicates!
Dataframe "stadia": Fixed dates!
Dataframe "stadia": Cleaned out 0 dates!
Lowered strings of cols ['afg_co', 'sta_code', 'sta_oms', 'afg_code_stad', 'afs_code', 'afs_oms', 'afg_code_afs', 'resultaat', 'mdr_code', 'user_created', 'user_modified', 'stadia_id', 'zaak_id', 'stadium_id', 'label'] in df stadia!
Missing values in df stadia have been imputed!


In [8]:
############################
## Clean personen dataset ##
############################

personenPipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=personenDataset.id_column,
        drop_duplicates=True,
        fix_date_columns=['geboortedatum'],
        lower_string_columns=True)
    )])

personen = personenPipeline.fit_transform(personenDataset.data)

Dataframe "personen": Dropped 0 duplicates!
Dataframe "personen": Fixed dates!
Lowered strings of cols ['pen_type', 'gezinsverhouding', 'geslacht', 'voorletters', 'burgerlijke_staat', 'naam', 'geheim_adres', 'voorv_mnaam', 'voorv_naam', 'meisjesnaam', 'vertrekdatum_adam', 'ind_naamgebruik', 'nat_ned', 'ind_nat_ovlp', 'verblijfstatus', 'datum_einde_vblstat', 'landcode', 'user_created', 'user_modified', 'datum_begin_vblstat', 'ais_nr', 'crv_nr', 'geheim', 'in_onderzoek', 'datum_verkrijging_vreemd', 'voorletters_zdia', 'naam_zdia', 'voorv_mnaam_zdia', 'voorv_naam_zdia', 'meisjesnaam_zdia', 'nm_dia_255', 'mnm_dia_255'] in df personen!
Missing values in df personen have been imputed!


In [9]:
####################################################
## Clean BAG dataset & perform feature extraction ##
####################################################

bagPipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=bagDataset.id_column,
        drop_duplicates=True,
        fix_date_columns=[],
        drop_columns = ['indicatie_geconstateerd', 'indicatie_in_onderzoek', 'woningvoorraad'],
        lower_string_columns=True,
        impute_missing_values=True,
        impute_missing_values_mode=['status_coordinaat_code'],
#         impute_missing_values_mode=['status_coordinaat_code', 'indicatie_geconstateerd',
#                                     'indicatie_in_onderzoek', 'woningvoorraad'],
        fillna_columns={'_huisnummer_verblijfsobject': 0,
                         '_huisletter_verblijfsobject': 'None',
                         '_openbare_ruimte_naam_verblijfsobject': 'None',
                         '_huisnummer_toevoeging_verblijfsobject': 'None',
                         'type_woonobject_omschrijving': 'None',
                         'eigendomsverhouding_id': 'None',
                         'financieringswijze_id': -1,
                         'gebruik_id': -1,
                         'reden_opvoer_id': -1,
                         'status_id_verblijfsobject': -1,
                         'toegang_id': 'None'})
    ),
#     ('extract', FeatureExtractionTransformer(
#         categorical_cols_hot=['status_coordinaat_code', 'type_woonobject_omschrijving',
#                               'eigendomsverhouding_id', 'financieringswijze_id',
#                               'gebruik_id', 'ligging_id', 'reden_opvoer_id',
#                               'status_id_nummeraanduiding', 'toegang_id'])
#     )
    ])

bagDataset.data = bagPipeline.fit_transform(bagDataset.data)
bag = bagDataset.data  # For easier usage.

Dataframe "bag": Dropped 0 duplicates!
Lowered strings of cols ['id_nummeraanduiding', 'landelijk_id_nummeraanduiding', 'huisletter_nummeraanduiding', 'huisnummer_toevoeging_nummeraanduiding', 'postcode', 'type', 'adres_nummer', 'vervallen_nummeraanduiding', 'hoofdadres', '_openbare_ruimte_naam_nummeraanduiding', 'bron_id_nummeraanduiding', 'ligplaats_id', 'openbare_ruimte_id', 'standplaats_id', 'status_id_nummeraanduiding', 'verblijfsobject_id', '_geom', 'id_ligplaats', 'landelijk_id_ligplaats', 'vervallen_ligplaats', 'geometrie_ligplaats', 'bron_id_ligplaats', 'status_id_ligplaats', 'id_standplaats', 'landelijk_id_standplaats', 'vervallen_standplaats', 'geometrie_standplaats', 'bron_id_standplaats', 'status_id_standplaats', 'id_verblijfsobject', 'landelijk_id_verblijfsobject', 'status_coordinaat_code', 'status_coordinaat_omschrijving', 'type_woonobject_code', 'type_woonobject_omschrijving', 'geometrie_verblijfsobject', '_huisletter_verblijfsobject', 'bron_id_verblijfsobject', 'eigend

In [10]:
###########################
## Clean hotline dataset ##
###########################

hotlinePipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=hotlineDataset.id_column,
        drop_duplicates=True,
        lower_string_columns=True,
        impute_missing_values=True)
    )])

hotlineDataset.data = hotlinePipeline.fit_transform(hotlineDataset.data)
hotline = hotlineDataset.data  # For easier usage.

Dataframe "hotline": Dropped 0 duplicates!
Lowered strings of cols ['mdw_code', 'overtreding_code', 'melder_anoniem', 'melder_naam', 'melder_emailadres', 'melder_telnr', 'situatie_schets', 'user_created', 'user_modified'] in df hotline!
Missing values in df hotline have been imputed!


In [11]:
#########################
## Clean adres dataset ##
#########################

# Hier de extract stap weghalen? Deze past waarschijnlijk beter na het combinen v/d datasets.
adresPipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=adresDataset.id_column,
        drop_duplicates=True,
        fix_date_columns=['hvv_dag_tek', 'max_vestig_dtm', 'wzs_update_datumtijd'],
        lower_string_columns=True,
        impute_missing_values=True,
        fillna_columns={'hsnr': 0, 'sttnaam': 'None', 'hsltr': 'None', 'toev': 'None'})
    )
])
#     ,
#     ('extract', FeatureExtractionTransformer(
#         categorical_cols_hot=['toev', 'pvh_omschr', 'sbw_omschr', 'sbv_omschr'],
#         ))
#     ])

adresDataset.data = adresPipeline.fit_transform(adresDataset.data)
adres = adresDataset.data  # For easier usage.

Dataframe "adres": Dropped 0 duplicates!
Dataframe "adres": Fixed dates!
Lowered strings of cols ['postcode', 'sdl_code', 'brt_code', 'pvh_cd', 'pvh_omschr', 'sbw_omschr', 'sbv_omschr', 'wzs_buurtcode_os_2015', 'wzs_buurtnaam_os_2015', 'wzs_buurtcombinatiecode_os_2015', 'wzs_buurtcombinatienaam_os_2015', 'wzs_22gebiedencode_os_2015', 'wzs_22gebiedennaam_os_2015', 'wzs_rayoncode_os_2015', 'wzs_rayonnaam_os_2015', 'wzs_stadsdeelcode_os_2015', 'wzs_stadsdeelnaam_os_2015', 'wzs_alternatieve_buurtennaam_os_2015', 'wzs_alternatieve_buurtencode_os_2015', 'wzs_geom', 'wzs_wijze_verrijking_geo', 'wzs_22gebiedencode_2015', 'wzs_22gebiedennaam_2015', 'sttnaam', 'hsltr', 'toev', 'brtcombi_naam', 'sdl_naam', 'brt_naam', 'a_dam_bag', 'landelijk_bag', 'hvv_dag_tek', 'max_vestig_dtm'] in df adres!
Missing values in df adres have been imputed!


# Combine the datasets

In [12]:
##########################
## Enrich adres dataset ##
##########################

# Enrich the adres dataset with information from the bag, personen and hotline datasets.
adresDataset.enrich_with_bag(bagDataset.data)
adresDataset.enrich_with_personen_features(personenDataset.data)
adresDataset.add_hotline_features(hotlineDataset.data)


##########################
## Enrich zaken dataset ##
##########################

# Only keep the finished cases in the zaken dataset (remove all unfinished cases).
zakenDataset.keep_finished_cases(stadiaDataset.data)

# Add a label to indicate woonfraude.
zakenDataset.add_binary_label_zaken(stadiaDataset.data)

Missing values in df adres have been imputed!
Saving version 'download_leegstand_woningId_bag' of dataframe 'adres'.


/home/tjongstra/.local/lib/python3.6/site-packages/pandas/core/generic.py:2378: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block1_values] [items->['postcode', 'sdl_code', 'brt_code', 'pvh_cd', 'pvh_omschr', 'sbw_omschr', 'sbv_omschr', 'wzs_buurtcode_os_2015', 'wzs_buurtnaam_os_2015', 'wzs_buurtcombinatiecode_os_2015', 'wzs_buurtcombinatienaam_os_2015', 'wzs_22gebiedencode_os_2015', 'wzs_22gebiedennaam_os_2015', 'wzs_rayoncode_os_2015', 'wzs_rayonnaam_os_2015', 'wzs_stadsdeelcode_os_2015', 'wzs_stadsdeelnaam_os_2015', 'wzs_alternatieve_buurtennaam_os_2015', 'wzs_alternatieve_buurtencode_os_2015', 'wzs_geom', 'wzs_wijze_verrijking_geo', 'wzs_22gebiedencode_2015', 'wzs_22gebiedennaam_2015', 'sttnaam', 'hsltr', 'toev', 'brtcombi_naam', 'sdl_naam', 'brt_naam', 'a_dam_bag', 'landelijk_bag', 'postcode_x', 'id_nummeraanduiding', 'landelijk_id_nummeraanduiding', 'huisletter_numme

The adres dataset is now enriched with BAG data.
Now looping over all address ids that have a link with one or more inhabitants...
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
Now looping over all rows in the adres dataframe in order to add person information...
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
...done!
Saving version 'download_leegstand_woningId_bag_personen' of dataframe 'adres'.


/home/tjongstra/.local/lib/python3.6/site-packages/pandas/core/generic.py:2378: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block5_values] [items->['postcode', 'sdl_code', 'brt_code', 'pvh_cd', 'pvh_omschr', 'sbw_omschr', 'sbv_omschr', 'wzs_buurtcode_os_2015', 'wzs_buurtnaam_os_2015', 'wzs_buurtcombinatiecode_os_2015', 'wzs_buurtcombinatienaam_os_2015', 'wzs_22gebiedencode_os_2015', 'wzs_22gebiedennaam_os_2015', 'wzs_rayoncode_os_2015', 'wzs_rayonnaam_os_2015', 'wzs_stadsdeelcode_os_2015', 'wzs_stadsdeelnaam_os_2015', 'wzs_alternatieve_buurtennaam_os_2015', 'wzs_alternatieve_buurtencode_os_2015', 'wzs_geom', 'wzs_wijze_verrijking_geo', 'wzs_22gebiedencode_2015', 'wzs_22gebiedennaam_2015', 'sttnaam', 'hsltr', 'toev', 'brtcombi_naam', 'sdl_naam', 'brt_naam', 'a_dam_bag', 'landelijk_bag', 'postcode_x', 'id_nummeraanduiding', 'landelijk_id_nummeraanduiding', 'huisletter_numme

The adres dataset is now enriched with personen data.
Saving version 'download_leegstand_woningId_bag_personen_hotline' of dataframe 'adres'.
The adres dataset is now enriched with hotline data.


/home/tjongstra/.local/lib/python3.6/site-packages/pandas/core/computation/expressions.py:183: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


Selected 16079 finished cases from a total of 31051 cases.
Saving version 'download_categories_filterCategories_finishedCases' of dataframe 'zaken'.


/home/tjongstra/.local/lib/python3.6/site-packages/pandas/core/generic.py:2378: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['afg_code_afs', 'afg_code_beh', 'afs_code', 'afs_oms', 'beh_code', 'beh_oms', 'categorie', 'eigenaar', 'mededelingen', 'zaak_id']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


Dataframe "zaken": added column "woonfraude" (binary label)


In [13]:
###############################################################################
## Remove implicit label columns and superfluous columns  from adres dataset ##
###############################################################################

adres_remove = [# Remove because cols do not exists when melding is received
                    'wzs_update_datumtijd',
                    # Remove because cols do not add extra information.
                    'kmrs',
                    'straatcode',
                    'xref',
                    'yref',
                    'postcode',
                    'wzs_buurtcode_os_2015',
                    'wzs_buurtcombinatiecode_os_2015',
                    'wzs_stadsdeelcode_os_2015',
                    'sttnaam',
                    'hvv_dag_tek', # Empty column
                    'max_vestig_dtm', # Empty column
                    'wzs_22gebiedencode_os_2015', # Empty column
                    'wzs_22gebiedennaam_os_2015', # Empty column
                    'sdl_naam',
                    'pvh_cd',
                    'sbv_code',
                    'sbw_code',
                    'wzs_wijze_verrijking_geo',
                    'wzs_22gebiedencode_2015',
                    'brt_naam',
                    'wzs_buurtnaam_os_2015',
                    'wzs_buurtcombinatienaam_os_2015',
                    'wzs_rayonnaam_os_2015',
                    'wzs_rayoncode_os_2015',
                    'wzs_stadsdeelnaam_os_2015',
                    'wzs_alternatieve_buurtennaam_os_2015',
                    'wzs_alternatieve_buurtencode_os_2015',
                    'hsltr',
                    'wzs_geom',
                    'brt_code',
                    'brtcombi_code',
                    'brtcombi_naam',
                    'sdl_code',
                    'wzs_22gebiedennaam_2015',
                    'wzs_id',
                    'a_dam_bag',
                    'landelijk_bag']

bag_remove = ['einde_geldigheid',               # Only 2 entries in column.
              'verhuurbare_eenheden',           # Only ~2k entries in column.
              'geometrie_ligplaats',            # Needs a lot of processing before being useful.
#               'bron_id',                      # Only 2 entries in column.
              'bron_id_verblijfsobject',        # Only 2 entries in column.
              'locatie_ingang_id',              # Only 2 entries in column.
              'reden_afvoer_id',                # Only a few entries in column.
              '_gebiedsgerichtwerken_id',       # Superfluous (gebied).
              '_grootstedelijkgebied_id',       # Superfluous (grootstedelijkgebied).
              'buurt_id',                       # Superfluous (buurt).
              # ONDERSTAANDE 4 KOLOMMEN KONDEN EERDER NIET WEG IVM MATCH MET ADRES DATAFRAME.
              # DEZE MOETEN NU WEL WEG, DAAROM WORDT NU HIER ALLES WEGGEHAALD.
              '_openbare_ruimte_naam_nummeraanduiding',          # Superfluous (straatnaam).
#               '_huisnummer_nummeraanduiding',                    # Superfluous (huisnummer).
#               '_huisletter_nummeraanduiding',                    # Superfluous (huisletter).
#               '_huisnummer_toevoeging_nummeraanduiding',         # Superfluous (huisnummer toevoeging).
#               'vervallen',                      # Superfluous (all values in col are equal).
              'vervallen_nummeraanduiding',
              'vervallen_ligplaats',
              'vervallen_standplaats',
              'vervallen_verblijfsobject',
              'document_mutatie',               # Not available at time of signal.
#               'date_modified',                  # Not available at time of signal.
              'date_modified_nummeraanduiding', # Not available at time of signal.
              'document_nummer',                # Not needed? (Swaan?)
              'status_coordinaat_omschrijving', # Not needed? (Swaan?)
              'type_woonobject_code',           # Not needed? (Swaan?)
              'id_ligplaats',                   # Not needed.
              'landelijk_id_ligplaats',         # Not needed.
              'id_standplaats',                 # Not needed.
              'landelijk_id_standplaats',       # Not needed.
              'id_verblijfsobject',             # Not needed.
              'landelijk_id_verblijfsobject',   # Not needed.
              ]

# Remove adres_id, since this is not a feature we want our algorihtm to try and learn from.
# adresDataset.data.drop(columns=adres_remove + bag_remove + ['adres_id'], inplace=True)
adresDataset.data.drop(columns=adres_remove + bag_remove, inplace=True)

In [14]:
####################################################
## Merge the adres dataset onto the zaken dataset ##
####################################################

# # Remove all unfinished cases in the zaken dataset.
# zakenDataset.keep_finished_cases(stadiaDataset.data)

# Merge the adres dataset onto the zaken dataset.
zakenDataset.data = zakenDataset.data.merge(adresDataset.data, on='adres_id', how='left')

In [15]:
# Als deze head zonder errors hierboven geprint wordt, dan werkt alles tot nu toe mooi.
zakenDataset.data.head()

,adres_id,afg_code_afs,afg_code_beh,afs_code,afs_oms,bedrag_huur,begindatum,beh_code,beh_oms,categorie,...,percentage_gezinsverhouding_3,gezinsverhouding_4,percentage_gezinsverhouding_4,gezinsverhouding_5,percentage_gezinsverhouding_5,gezinsverhouding_6,percentage_gezinsverhouding_6,gezinsverhouding_7,percentage_gezinsverhouding_7,aantal_hotline_meldingen
0,44312.0,vtbd,vtbd,pwibg,zl woning is beschikbaar gekomen,466.845141,2012-09-25,zkl_dr_pm,zkl doorzon prostitutie/mensenhandel,prostitutie,...,0.0,0,0.0,0,0.0,4,1.0,0,0.0,0
1,1889860.0,vtbd,vtbd,pwibg,zl woning is beschikbaar gekomen,750.000000,2014-03-05,zkl_dr_drg,zkl doorzon drugs,drugs,...,0.0,0,0.0,2,0.5,0,0.0,0,0.0,0
2,320561.0,vtbd,vtbd,pgwg,zl geen woonfraude,546.000000,2013-01-14,zkl_dr_ol,zkl doorzon onderhuur/leegstand,illegale onderhuur,...,0.5,0,0.0,0,0.0,1,0.5,0,0.0,0
3,230853.0,vtbd,vtbd,pwibg,zl woning is beschikbaar gekomen,757.000000,2014-01-15,zkl_dr_hos,zkl doorzon illegaal hotel/shortstay,illegale hotels,...,0.0,0,0.0,0,0.0,4,1.0,0,0.0,2
4,307169.0,vtbd,vtbd,pgwg,zl geen woonfraude,465.000000,2013-09-27,zkl_dr_pm,zkl doorzon prostitutie/mensenhandel,prostitutie,...,0.2,0,0.0,0,0.0,2,0.4,0,0.0,0


# Perform  Feature Extraction

In [16]:
#################################################
## Perform feature extraction on zaken dataset ##
#################################################

categorical_col_hot_zaken = ['afg_code_beh', 'beh_code', 'eigenaar', 'categorie']
categorical_cols_hot_adres = ['toev', 'pvh_omschr', 'sbw_omschr', 'sbv_omschr']
categorical_cols_hot_bag = ['status_coordinaat_code', 'type_woonobject_omschrijving',
                            'eigendomsverhouding_id', 'financieringswijze_id',
                            'gebruik_id', 'ligging_id', 'reden_opvoer_id',
                            'status_id_nummeraanduiding', 'toegang_id']

zakenPipeline = Pipeline(steps=[
    ('extract', FeatureExtractionTransformer(
        categorical_cols_hot=categorical_col_hot_zaken + categorical_cols_hot_adres + categorical_cols_hot_bag,
        ))
    ])

zakenDataset.data = zakenPipeline.fit_transform(zakenDataset.data)

Now extracting features from column: 'afg_code_beh'.
Done!
Now extracting features from column: 'beh_code'.
Done!
Now extracting features from column: 'eigenaar'.
Done!
Now extracting features from column: 'categorie'.
Done!
Now extracting features from column: 'toev'.
Done!
Now extracting features from column: 'pvh_omschr'.
Done!
Now extracting features from column: 'sbw_omschr'.
Done!
Now extracting features from column: 'sbv_omschr'.
Done!
Now extracting features from column: 'status_coordinaat_code'.
Done!
Now extracting features from column: 'type_woonobject_omschrijving'.
Done!
Now extracting features from column: 'eigendomsverhouding_id'.
Done!
Now extracting features from column: 'financieringswijze_id'.
Done!
Now extracting features from column: 'gebruik_id'.
Done!
Now extracting features from column: 'ligging_id'.
Done!
Now extracting features from column: 'reden_opvoer_id'.
Done!
Now extracting features from column: 'status_id_nummeraanduiding'.
Done!
Now extracting features

# Save/Load Finalized Dataset

In [17]:
# Save.
zakenDataset.version = 'final'
zakenDataset.save()

Saving version 'final' of dataframe 'zaken'.


/home/tjongstra/.local/lib/python3.6/site-packages/pandas/core/generic.py:2378: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block1_values] [items->['afg_code_afs', 'afs_code', 'afs_oms', 'beh_oms', 'mededelingen', 'zaak_id', 'postcode_x', 'id_nummeraanduiding', 'landelijk_id_nummeraanduiding', 'huisletter_nummeraanduiding', 'huisnummer_toevoeging_nummeraanduiding', 'type', 'adres_nummer', 'bron_id_nummeraanduiding', 'ligplaats_id', 'openbare_ruimte_id', 'standplaats_id', 'verblijfsobject_id', '_geom', 'bron_id_ligplaats', 'status_id_ligplaats', 'geometrie_standplaats', 'bron_id_standplaats', 'status_id_standplaats', 'geometrie_verblijfsobject', '_huisletter_verblijfsobject', 'status_id_verblijfsobject']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [86]:
# Load.
zakenDataset = ZakenDataset()
zakenDataset.load('final')

Version 'final' of dataset 'zaken' loaded!


# Show Dataset Statistics

In [87]:
# Show percentage of positive samples in dataset.
print(f"Number of entries: {len(zakenDataset.data)}")
print(f"Percentage positives: {round((zakenDataset.data.woonfraude.sum() * 100) / len(zakenDataset.data.woonfraude), 1)}%")

Number of entries: 16079
Percentage positives: 53.3%


# Create Train/Test Split

In [88]:
# Remove the adres_id column.
zakenDataset.data.drop(columns=['adres_id'], inplace=True)

# Remove text columns, which can't be used for training.
zakenDataset.data.drop(columns=['afg_code_afs', 'afs_code', 'afs_oms', 'beh_oms', 'mededelingen'], inplace=True)

# Only keep numeric data columns.
zakenDataset.data = zakenDataset.data._get_numeric_data()

# Remove columns containing only NaN values.
zakenDataset.data.drop(columns=['hoofdadres', 'begin_geldigheid'], inplace=True)

In [90]:
# Split up the dataset (only use numeric data!).
X_train, X_test, y_train, y_test = split_data_train_test(zakenDataset.data)

Original dataset shape Counter({True: 8572, False: 7507})
Training set shape Counter({True: 7286, False: 6381})
Testing set shape Counter({True: 1286, False: 1126})


/usr/local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


# Train Model

In [107]:
# Utility function to report best scores.
def report(results, n_top=10):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [108]:
# Specify parameters and distributions to sample from.
param_dist = {
              "n_estimators": sp_randint(100, 2000),
#               "max_features": ['auto'],
              "max_features": sp_randint(1, 100),
              "max_depth": sp_randint(1, 100),
              "min_samples_leaf": [1],
              "min_samples_split": sp_randint(2, 5),
              "bootstrap": [True, False],
#               "criterion": ["gini", "entropy"],
             }

In [109]:
# Run randomized search.
clf = RandomForestClassifier()
n_iter = 100
random_search = RandomizedSearchCV(clf,
                                   param_distributions=param_dist,
                                   n_iter=n_iter,
                                   cv=10,
                                   n_jobs=-1,
                                   scoring='f1')

start = time.time()
random_search.fit(X_train, y_train)

# Print results.
print(f"RandomizedSearchCV took {time.time() - start} seconds for {len(random_search.cv_results_['params'])} candidate parameter settings.")
report(random_search.cv_results_)

/usr/local/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


RandomizedSearchCV took 16673.81831240654 seconds for 100 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.767 (std: 0.006)
Parameters: {'bootstrap': False, 'max_depth': 16, 'max_features': 74, 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 1970}

Model with rank: 2
Mean validation score: 0.766 (std: 0.007)
Parameters: {'bootstrap': True, 'max_depth': 21, 'max_features': 61, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 483}

Model with rank: 3
Mean validation score: 0.766 (std: 0.006)
Parameters: {'bootstrap': True, 'max_depth': 34, 'max_features': 58, 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 1500}

Model with rank: 4
Mean validation score: 0.766 (std: 0.004)
Parameters: {'bootstrap': True, 'max_depth': 34, 'max_features': 57, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 1554}

Model with rank: 5
Mean validation score: 0.766 (std: 0.006)
Parameters: {'bootstrap': True, 'max_depth': 32, 'm

In [ ]:
# Save best model for later reuse in dashboard. Manually put this model in the "data" folder (temporary solution).
best_random_forest_classifier_temp = random_search.best_estimator_
pickle.dump(best_random_forest_classifier_temp, open("best_random_forest_classifier_temp.pickle", "wb"))